In [8]:
import pandas as pd
from dataclasses import dataclass

@dataclass
class CoinPair:
    CoinA: str
    CoinB: str
    
max_std = 0.1
min_profit = 0.2

pairs = pd.read_csv('Output/best_pairs.csv', index_col=[0,1])



filt_idx = pairs[(pairs['std'] < max_std) & (pairs['profit'] > min_profit)].sort_values(by='profit', ascending=False)

pairs_dataClass = [CoinPair(r[0], r[1]) for r in filt_idx]


filt_idx

profit       std  total_trades
CoinA     CoinB                                      
AIXBTUSDT SEIUSDT    0.324933  0.056882             8
          THETAUSDT  0.314666  0.059350             6
          SAGAUSDT   0.305360  0.047462            16
          GRTUSDT    0.274298  0.063056             8
          ZENUSDT    0.233826  0.069127             4
CVCUSDT   STEEMUSDT  0.232920  0.020089            18
AIXBTUSDT ACEUSDT    0.218976  0.054552            18
          ORDIUSDT   0.201431  0.063657            16

In [ ]:

def trainSymbols(candles: pd.DataFrame, highCorr_500: pd.DataFrame,
                 max_pairs, max_hold, 
                 step, r1 = 10, r2=15, l = 5, roll_window = 30, 
                 train_rows_start = 1000, train_rows_end = 2200):
    
    if max_pairs != 'all':
        symbols = highCorr_500['CoinA'].sample(max_pairs).unique().tolist()
        print('Total Amount of Unique Symbols:', len(symbols))
        
    else:
        symbols = highCorr_500['CoinA'].unique().tolist()
        print('Total Amount of Unique Symbols:', len(symbols))

    

    dfs = []
    last_training_end = None
    tasks = []

    for symbol in symbols:
        current_start = train_rows_start
        while True:
            next_start = current_start + step
            if next_start >= train_rows_end:
                break

            traincandles = candles[current_start:next_start].copy()
            testcandles = candles[next_start: next_start + step].reset_index().rename(columns = {'date':'time'}).copy()
            highCorr_ = highCorr_500[highCorr_500['CoinA'] == symbol]
            tasks.append((traincandles, testcandles, highCorr_, dfs, r1, r2, l, max_hold, roll_window))


            last_training_end = next_start + step
            current_start = next_start
            

    total = len(tasks)
    pbar = tqdm(total=total, dynamic_ncols=True, mininterval=0.1, smoothing=0.3)

    with ThreadPoolExecutor(max_workers=os.cpu_count()) as ex:  # cap concurrency (tune 4–16)
        futures = [ex.submit(train, *args) for args in tasks]
        for fut in as_completed(futures):
            try:
                fut.result()   # your train() appends into dfs
            finally:
                pbar.update(1)

    pbar.close()

In [ ]:
import re


text ="""[Errno 2] No such file or directory: 'CryptoTest'
/content/CryptoTest
10000
Total Amount of Unique Symbols: 56
100% 56/56 [05:29<00:00,  5.89s/it]
--- Final Results --- 

Total Profit: 0.030284621365244423
Mean Profit per iteration: 0.015142310682622201
Max Standard Deviation of All Pairs: 0.15265308009331732
Total Trades: 214
Max Hold Time: 10
Total Amount of Unique Symbols: 56
100% 56/56 [02:33<00:00,  2.74s/it]
--- Final Results --- 

Total Profit: 4.546132115616127
Mean Profit per iteration: 0.5051257906240142
Max Standard Deviation of All Pairs: 0.27837417650726853
Total Trades: 1112
Max Hold Time: 10
Total Amount of Unique Symbols: 68
100% 68/68 [22:41<00:00, 20.03s/it]
--- Final Results --- 

Total Profit: 1.173704482090781
Mean Profit per iteration: 0.03452072006149356
Max Standard Deviation of All Pairs: 0.17068485480085815
Total Trades: 986
Max Hold Time: 10
Total Amount of Unique Symbols: 81
100% 81/81 [34:59<00:00, 25.92s/it]
--- Final Results --- 

Total Profit: 0.16188367234353235
Mean Profit per iteration: 0.04047091808588309
Max Standard Deviation of All Pairs: 0.3858174656169052
Total Trades: 404
Max Hold Time: 10
Total Amount of Unique Symbols: 84
100% 84/84 [49:04<00:00, 35.05s/it]
--- Final Results --- 

Total Profit: -2.171236851185987
Mean Profit per iteration: -0.08041617967355509
Max Standard Deviation of All Pairs: 0.29108761518336546
Total Trades: 950
Max Hold Time: 10
Total Amount of Unique Symbols: 88
100% 88/88 [25:00<00:00, 17.05s/it]
--- Final Results --- 

Total Profit: 0.7194276329689694
Mean Profit per iteration: 0.35971381648448475
Max Standard Deviation of All Pairs: 0.2990660521011468
Total Trades: 170
Max Hold Time: 10
Total Amount of Unique Symbols: 85
100% 85/85 [26:43<00:00, 18.86s/it]
--- Final Results --- 

Total Profit: -1.22449032169924
Mean Profit per iteration: -0.061224516084962
Max Standard Deviation of All Pairs: 0.26074149680833575
Total Trades: 1036
Max Hold Time: 10
Total Amount of Unique Symbols: 84
100% 84/84 [55:45<00:00, 39.83s/it]
--- Final Results --- 

Total Profit: 2.0862043816553384
Mean Profit per iteration: 0.1604772601273337
Max Standard Deviation of All Pairs: 0.22855884544976582
Total Trades: 946
Max Hold Time: 10"""



test = list(float(nr) for nr in re.findall(r'Total Profit: ([-+]?\d+\.\d+)', text))

sum(test)

5.321909733154765